In [8]:
pip install pennylane --upgrade

  Using cached PennyLane-0.33.1-py3-none-any.whl (1.5 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 72.6 MB/s eta 0:00:00


In [9]:
import numpy as np
import pandas as pd
from sklearn import datasets
import seaborn as sns
import jax;
jax.config.update('jax_platform_name', 'cpu')
jax.config.update("jax_enable_x64", True)
import jax.numpy as jnp
import optax  # optimization using jax
import pennylane as qml
import pennylane.numpy as pnp

In [10]:
seed=0
rng=np.random.default_rng(seed=seed)
Num_wires=8
Device=qml.device("default.qubit", wires=Num_wires)

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [24]:
def Read_MNIST(data):
 fr=open("/content/drive/MyDrive/Colab Notebooks/Quantum machine learning/2023 Ephys Challenge/CIFAR-100_%s_16_16.text"%(data),"r")
 img=[]
 trg=[]
 for line in fr:
  lx=line.split()
  if len(lx)==1:
   trg.append(int(lx[0]))
  else:
   for i in range (len(lx)):
    img.append(np.float64(lx[i]))
 fr.close()
 img2=np.asarray(img).reshape(-1,16*16)
 img3=img2/np.linalg.norm(img2,axis=1).reshape((-1,1))
 return jnp.asarray(trg),jnp.asarray(img3)

In [25]:
def Read_Data():
 y_test,x_test=Read_MNIST("test")
 y_train,x_train=Read_MNIST("train")
 return x_train,y_train,x_test,y_test

In [26]:
def convolutional_layer(weights,wires,skip_first_layer=True):
 n_wires=len(wires)
 assert n_wires >= 3, "this circuit is too small!"
 for p in [0, 1]:
  for indx, w in enumerate(wires):
   if indx % 2 == p and indx < n_wires - 1:
    if indx % 2 == 0 and not skip_first_layer:
      qml.U3(*weights[:3],wires=[w])
      qml.U3(*weights[3:6],wires=[wires[indx+1]])
    qml.IsingXX(weights[6],wires=[w,wires[indx+1]])
    qml.IsingYY(weights[7],wires=[w,wires[indx+1]])
    qml.IsingZZ(weights[8],wires=[w,wires[indx+1]])
    qml.U3(*weights[9:12],wires=[w])
    qml.U3(*weights[12:],wires=[wires[indx+1]])

In [27]:
def pooling_layer(weights, wires):
 n_wires = len(wires)
 assert len(wires) >= 2, "this circuit is too small!"
 for indx, w in enumerate(wires):
  if indx % 2 == 1 and indx < n_wires:
   m_outcome = qml.measure(w)
   qml.cond(m_outcome, qml.U3)(*weights, wires=wires[indx - 1])

In [28]:
def conv_and_pooling(kernel_weights,n_wires,skip_first_layer=True):
 convolutional_layer(kernel_weights[:15],n_wires,skip_first_layer=skip_first_layer)
 pooling_layer(kernel_weights[15:],n_wires)

In [29]:
def dense_layer(weights,wires):
 qml.ArbitraryUnitary(weights,wires)

In [30]:
@qml.qnode(Device,interface="jax")
def conv_net(weights,last_layer_weights,features):
 layers=weights.shape[1]
 wires=list(range(Num_wires))
 qml.AmplitudeEmbedding(features=features,wires=wires,pad_with=0.5)
 qml.Barrier(wires=wires,only_visual=True)
 for j in range(layers):
  conv_and_pooling(weights[:,j],wires,skip_first_layer=(not j == 0))
  wires = wires[::2]
  qml.Barrier(wires=wires, only_visual=True)
 assert last_layer_weights.size == 4 ** (len(wires)) - 1, (
  "The size of the last layer weights vector is incorrect!"
  f" \n Expected {4 ** (len(wires)) - 1}, Given {last_layer_weights.size}")
 dense_layer(last_layer_weights, wires)
 return qml.probs(wires=(0))

In [31]:
@jax.jit
def compute_out(weights, weights_last, features, labels):
  cost=lambda weights,weights_last,feature,label:conv_net(weights,weights_last,feature)[label]
  return jax.vmap(cost,in_axes=(None,None,0,0),out_axes=0)(weights,weights_last,features,labels)
def compute_accuracy(weights, weights_last, features, labels):
  out=compute_out(weights,weights_last,features,labels)
  return jnp.sum(out>0.5)/len(out)
def compute_cost(weights,weights_last,features,labels):
  out=compute_out(weights,weights_last,features,labels)
  return 1.0-jnp.sum(out)/len(labels)
def init_weights():
  weights=pnp.random.normal(loc=0,scale=1,size=(18,2),requires_grad=True) #18參數 2層
  weights_last=pnp.random.normal(loc=0,scale=1,size=4**2-1,requires_grad=True)
  return jnp.array(weights),jnp.array(weights_last)
value_and_grad=jax.jit(jax.value_and_grad(compute_cost,argnums=[0,1]))

In [32]:
def train_qcnn(n_train,n_test,n_epochs):
 x_train,y_train,x_test,y_test=Read_Data()
 weights,weights_last=init_weights()
 cosine_decay_scheduler=optax.cosine_decay_schedule(0.1,decay_steps=n_epochs,alpha=0.95)
 optimizer=optax.adam(learning_rate=cosine_decay_scheduler)
 opt_state=optimizer.init((weights,weights_last))
 train_cost_epochs,test_cost_epochs,train_acc_epochs,test_acc_epochs=[],[],[],[]
 for step in range(n_epochs):
  train_cost,grad_circuit=value_and_grad(weights,weights_last,x_train,y_train)
  updates,opt_state=optimizer.update(grad_circuit,opt_state)
  weights,weights_last=optax.apply_updates((weights,weights_last),updates)
  train_cost_epochs.append(train_cost)
  train_acc=compute_accuracy(weights,weights_last,x_train,y_train)
  train_acc_epochs.append(train_acc)
  test_out=compute_out(weights,weights_last,x_test,y_test)
  test_acc=jnp.sum(test_out>0.5)/len(test_out)
  test_acc_epochs.append(test_acc)
  test_cost=1.0-jnp.sum(test_out)/len(test_out)
  test_cost_epochs.append(test_cost)
 return train_cost_epochs,test_cost_epochs,train_acc_epochs,test_acc_epochs

In [33]:
def Write_acc(train_cost,test_cost,train_acc,test_acc):
 fw=open("/content/drive/MyDrive/Colab Notebooks/Quantum machine learning/2023 Ephys Challenge/accuracy_16_16.text","w")
 fw.write("epoch train_cost  test_cost  train_acc   test_acc\n")
 for i in range (len(train_acc)):
  fw.write("%5d %10.2f %10.2f %10.2f %10.2f\n"%(i,train_cost[i],test_cost[i],train_acc[i],test_acc[i]))
 fw.close()

In [48]:
def Main():
 n_train=10000
 n_test=3000
 n_epochs=30
 train_cost,test_cost,train_acc,test_acc=train_qcnn(n_train,n_test,n_epochs)
 Write_acc(train_cost,test_cost,train_acc,test_acc)

In [49]:
Main()